In [0]:
dbutils.widgets.text("proc_date", "", "")
proc_date = dbutils.widgets.get("proc_date")

In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_store
TBLPROPERTIES (
'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
plant0 AS (
    SELECT * EXCEPT(rn)
    FROM (
        SELECT hash_id, plant, country, distr_chan, factcal_id, 
    IFNULL(CASE WHEN CAST(latitude AS DOUBLE) = CAST(CAST(latitude AS DOUBLE) AS BIGINT) THEN CAST(CAST(latitude AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(latitude AS DECIMAL(38,10)))) END, "") AS latitude,
    loc_currcy,
    IFNULL(CASE WHEN CAST(longitude AS DOUBLE) = CAST(CAST(longitude AS DOUBLE) AS BIGINT) THEN CAST(CAST(longitude AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(longitude AS DECIMAL(38,10)))) END, "") AS longitude,
    plantcat,
    postal_cd, postcd_gis, purch_org, region, rt_custpl, salesorg, sales_dist, comp_code,
    _bic_zc_sup_rg, _bic_zc_region, _bic_zc_saunit, segment,
    IFNULL(CASE WHEN CAST(_bic_zk_saqty AS DOUBLE) = CAST(CAST(_bic_zk_saqty AS DOUBLE) AS BIGINT) THEN CAST(CAST(_bic_zk_saqty AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(_bic_zk_saqty AS DECIMAL(38,10)))) END, "") AS _bic_zk_saqty,
    IFNULL(CASE WHEN CAST(_bic_zk_reaqty AS DOUBLE) = CAST(CAST(_bic_zk_reaqty AS DOUBLE) AS BIGINT) THEN CAST(CAST(_bic_zk_reaqty AS BIGINT) AS STRING) ELSE TRIM(TRAILING '0' FROM(CAST(_bic_zk_reaqty AS DECIMAL(38,10)))) END, "") as _bic_zk_reaqty,
    _bic_zk_numflr, currency, _bic_zk_renamt, _bic_zc_stoloc, crm_delblk,_bic_zc_catsiz,
    _bic_zc_posout, _bic_zc_posin, _bic_zc_rptsgm, _bic_zc_catage, _bic_zc_stosts, _bic_zc_kpists,
    street, name, name2, city, email_addr, addr_numbr, fax_number, tel_number, _bic_zc_sitel2,
    _bic_zc_sitel3, _bic_zc_gplant, _bic_zc_dcgr01, _bic_zc_subrg1, proc_date, rt_lbldatf,
    rt_lbldatt, rt_lcldat, rt_lopdat, aedat, start_date, end_date, file_creation_ts, ROW_NUMBER() OVER (PARTITION BY plant ORDER BY end_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_0plant
        WHERE segment IN ('1009', '1100', '1500', '2000') OR REGEXP_LIKE(plant,r'F[0-9]')
    ) a WHERE rn = 1
),
plantt AS (
    SELECT * EXCEPT(rn)
    FROM (
        SELECT *, ROW_NUMBER() OVER (PARTITION BY plant ORDER BY end_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_tplant
    ) a WHERE rn = 1
),
store_operation AS (
    SELECT * EXCEPT(rn)
    FROM (
        SELECT
            store_id, store_name, cua_hang_truong, sdt_cht, sdt_ch, email_ch, qlkv, sdt_qlkv, email_qlkv, gdv, email_gdv, gdm, email_gdm, ma_vg vm_region, ebitda ebitda_group,
            hub_hang_kho,
            ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY proc_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_rop.store_operation
    ) a WHERE rn = 1
),
store_status_wp AS (
    SELECT * EXCEPT(rn)
    FROM (
        SELECT
            CAST(store_id AS STRING) store_id,
            store_name,
            validate_from,
            validate_to,
            reason,
            `comment`,
            updated_date,
            updated_user,
            DATE(proc_date) last_updated_date,
            ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY proc_date DESC) rn
        FROM {catalog_name}.udp_wcm_silver_rop.store_status
    ) WHERE rn = 1
),
store_wp AS (
    SELECT
        a.* EXCEPT(rn, proc_date),
        DATE(proc_date) last_updated_date
    FROM (
        SELECT
              store_id,
              store_name,
              business_unit,
              sap_pc,
              region,
              city,
              district,
              ward,
              address,
              longtitude,
              latitude,
              open_date,
              ground,
              front_m,
              area_m2,
              biz_area_m2,
              size_v,
              assortment_v,
              concept_id,
              CASE TRY_CAST(concept AS BIGINT)
                WHEN 1 THEN 'Trung tâm - Lõi dân cư'
                WHEN 2 THEN 'Chung cư cao cấp'
                WHEN 3 THEN 'Chung cư trung cấp'
                WHEN 6 THEN 'WinStore'
                WHEN 7 THEN 'Minimart Base'
                WHEN 8 THEN 'Minimart Fresh'
                WHEN 9 THEN 'Minimart FullFresh'
                WHEN 14 THEN 'Fresh'
                WHEN 11 THEN 'Rural'
                WHEN 12 THEN 'Premium Fresh'
                WHEN 13 THEN 'WiN'
                WHEN 16 THEN 'Hub'
                ELSE concept
              END AS concept,
              warehouse_size,
              lease_term,
              status,
              store_type,
              proc_date,
              ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY proc_date DESC) rn
          FROM {catalog_name}.udp_wcm_silver_rop.store_profile_new
    ) a WHERE rn = 1
),
store_wp_not_in_plan AS (
    SELECT a.*
    ,'' AS `comment`
    FROM store_wp a
        LEFT JOIN store_status_wp b ON a.store_id = b.store_id
    WHERE a.store_id NOT IN (
        SELECT store_id
        FROM store_status_wp
        WHERE validate_from >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
        --
        AND reason IN ('3', '5')
    )
)
,store_wp_in_plan AS (
    SELECT
        a.store_id,
        a.store_name,
        b.business_unit,
        b.sap_pc,
        b.region,
        b.city,
        b.district,
        b.ward,
        b.address,
        b.longtitude,
        b.latitude,
        IF(a.reason = '5', validate_from, b.open_date) AS open_date,
        b.ground,
        b.front_m,
        b.area_m2,
        b.biz_area_m2,
        b.size_v,
        b.assortment_v,
        b.concept_id,
        b.concept,
        b.warehouse_size,
        b.lease_term,
        CASE
            WHEN a.reason = '3' THEN 'Sắp đóng cửa'
            WHEN a.reason = '5' THEN 'Sắp khai trương'
        END status,
        CAST(NULL AS STRING) store_type,
        a.last_updated_date,
        CONCAT(a.comment, '. Từ ngày: ', a.validate_from, ' đến ngày: ', a.validate_to, '. Ngày update ', a.updated_date) AS `comment`
    FROM store_status_wp a
    LEFT JOIN store_wp b ON a.store_id = b.store_id
    WHERE validate_from >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AND reason IN ('3', '5')
),
store_wp_all AS (
    SELECT * FROM store_wp_not_in_plan
    UNION ALL
    SELECT * FROM store_wp_in_plan
),
store_sap AS (
    SELECT
        a.* EXCEPT(name, city, street, _bic_zk_saqty, _bic_zk_reaqty),
        IF(a._bic_zk_reaqty < a._bic_zk_saqty AND a.plant = '1666', a._bic_zk_reaqty, a._bic_zk_saqty) _bic_zk_saqty,
        IF(a._bic_zk_reaqty < a._bic_zk_saqty AND a.plant = '1666', a._bic_zk_saqty, a._bic_zk_reaqty) _bic_zk_reaqty,
        b.txtsh name,
        b.txtmd store_name,
        IF(c.address IS NOT NULL, c.address, a.street) address,
        IF(c.city IS NOT NULL, c.city, a.city) city,
        DATE(a.proc_date) last_updated_date,
        CASE _bic_zc_dcgr01
          WHEN 'UB' THEN 'Urban'
          WHEN 'RR' THEN 'Rural'
          WHEN 'WL' THEN 'WiN'
          WHEN 'WT' THEN 'WMT'
        ELSE NULL
        END sap_concept
    FROM plant0 a
    LEFT JOIN plantt b ON a.plant = b.plant
    LEFT JOIN (
        SELECT
            addr_numbr,
            CONCAT(IFNULL(street60,''), ' ', IFNULL(str_suppl2,''), ' ', IFNULL(city_2,''), ' ', IFNULL(city_1,'')) address,
            city_1 city
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_paddr_numbr c
        WHERE end_date = '2400-01-01'
        ) c 
        ON LTRIM('0', CAST(a.addr_numbr AS STRING)) = c.addr_numbr
        WHERE UPPER(txtmd) NOT LIKE '%[DO NOT USE]%'
),
dc_fc AS (
    SELECT
        a.plant store_id,
        a.store_name,
        a.store_name store_description,
        rt_lopdat store_open_date,
        rt_lcldat store_closed_date,
        'FCDC' AS business_unit,
        a.region,
        a.region region_vn,
        a.city city_vn,
        '' district,
        '' ward,
        a.address,
        a.addr_numbr,
        '' sap_pc,
        '' cua_hang_truong,
        '' sdt_cht,
        '' sdt_ch,
        '' email_ch,
        '' qlkv,
        '' sdt_qlkv,
        '' email_qlkv,
        '' gdv,
        '' email_gdv,
        '' gdm,
        '' email_gdm,
        CAST(a.latitude AS STRING) latitude,
        CAST(a.longitude AS STRING) longtitude,
        '' ground,
        '' front_m,
        CAST(_bic_zk_reaqty AS STRING) area_m2,
        CAST(_bic_zk_saqty AS STRING) biz_area_m2,
        CAST(_bic_zc_catsiz AS STRING) size_v,
        '' assortment_v,
        '' concept_id,
        '' concept,
        a.sap_concept,
        '' warehouse_size,
        '' lease_term,
        a.comp_code,
        a.salesorg sales_region,
        a.sales_dist sales_district,
        a.distr_chan distribution_channel,
        a._bic_zc_sup_rg supply_region,
        a._bic_zc_rptsgm reporting_business_unit,
        a._bic_zk_saqty sales_area,
        a._bic_zk_reaqty rent_area,
        a._bic_zk_renamt rent_amount,
        a._bic_zc_region reporting_region,
        a._bic_zc_stoloc store_cat,
        a._bic_zc_catsiz store_cat_size,
        a._bic_zc_sitel2 site_group_l2,
        a._bic_zc_sitel3 site_group_l3,
        a._bic_zc_subrg1 sub_region_1,
        '' vm_region,
        a._bic_zc_kpists kpi_flag,
        CASE
            WHEN a._bic_zk_saqty < 1000 THEN 'A'
            WHEN a._bic_zk_saqty >= 1000 AND a._bic_zk_saqty < 1500 THEN 'B'
            WHEN a._bic_zk_saqty >= 1500 THEN 'C'
        ELSE NULL
        END size_st,
        CASE
            WHEN a._bic_zk_saqty < 1000 THEN '< 1.000 m2'
            WHEN a._bic_zk_saqty >= 1000 AND a._bic_zk_saqty < 1500 THEN '1.000 m2 - <1.500 m2'
            WHEN a._bic_zk_saqty >= 1500 THEN '>= 1.500 m2'
        ELSE NULL
        END sales_area_text,
        '' ebitda_group,
        '' hub_hang_kho,
        IF(store_name LIKE '[Block]%' AND store_name LIKE '%Adayroi%', 'Đóng cửa', 'Đang hoạt động') store_status_desc,
        '' comment,
        a.last_updated_date
    FROM store_sap a
    WHERE REGEXP_LIKE(a.plant,r'F[0-9]') AND store_name LIKE '%FWMP%'
),
SUPPLY_STORE_FC AS (

    SELECT a.customer ref_fc_code, b.oi_sort1 store_id, b.oi_sort2 supply_store 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_pcustomer a 
    JOIN {catalog_name}.udp_wcm_silver_sap_bw.sap_pzc_add_cl b
    ON a.addr_numbr = b._bic_zc_add_cl WHERE a.addr_numbr IS NOT NULL AND REGEXP_LIKE(b.oi_sort1,r'F[0-9]')
    AND a.accnt_grp = 'Y998'
    UNION ALL
    SELECT a.customer ref_fc_code, /*b.oi_sort1*/ a.customer store_id, b.oi_sort2 supply_store 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_pcustomer a 
    JOIN {catalog_name}.udp_wcm_silver_sap_bw.sap_pzc_add_cl b
    ON a.addr_numbr = b._bic_zc_add_cl WHERE a.addr_numbr IS NOT NULL
    AND a.customer BETWEEN '6A00' AND '6Z99'
    AND  b.oi_sort2 IS NOT NULL
),
store AS (
    SELECT
        a.store_id,
        a.store_name,
        CASE
            WHEN LENGTH(b.store_name) > LENGTH(a.store_name) THEN b.store_name
            ELSE a.store_name
        END store_description,
        a.open_date store_open_date,
        IF(b.rt_lcldat = '9999-12-31', NULL, b.rt_lcldat) store_closed_date,
        CASE
            WHEN a.business_unit = 'Vinmart' THEN '1500'
            WHEN a.business_unit = 'Vinmart+' THEN '2000'
            WHEN a.business_unit = 'Franchise' THEN '6000'
        ELSE b.segment
        END business_unit,
        IF(a.business_unit = 'Franchise', b2.region, b.region) region,
        a.region region_vn,
        IFNULL(a.city, b.city) city_vn,
        a.district,
        a.ward,
        IF(a.address IS NOT NULL, a.address, b.address) address,
        b.addr_numbr,
        a.sap_pc,
        c.cua_hang_truong,
        c.sdt_cht sdt_cht,
        c.sdt_ch sdt_ch,
        c.email_ch email_ch,
        c.qlkv,
        c.sdt_qlkv,
        c.email_qlkv,
        c.gdv,
        c.email_gdv,
        c.gdm,
        c.email_gdm,
        IFNULL(a.latitude, CAST(b.latitude AS STRING)) latitude,
        IFNULL(a.longtitude, CAST(b.longitude AS STRING)) longtitude,
        a.ground,
        a.front_m,
        IFNULL(a.area_m2, CAST(_bic_zk_reaqty AS STRING)) area_m2,
        IFNULL(a.biz_area_m2, CAST(_bic_zk_saqty AS STRING)) biz_area_m2,
        IFNULL(a.size_v, _bic_zc_catsiz) size_v,
        a.assortment_v,
        a.concept_id,
        a.concept,
        b.sap_concept,
        a.warehouse_size,
        a.lease_term,
        b.comp_code,
        b.salesorg sales_region,
        b.sales_dist sales_district,
        b.distr_chan distribution_channel,
        CASE WHEN a.business_unit = 'Franchise' THEN b2.supply_region ELSE b._bic_zc_sup_rg END supply_region,
        b._bic_zc_rptsgm reporting_business_unit,
        b._bic_zk_saqty sales_area,
        b._bic_zk_reaqty rent_area,
        b._bic_zk_renamt rent_amount,
        b._bic_zc_region reporting_region,
        b._bic_zc_stoloc store_cat,
        b._bic_zc_catsiz store_cat_size,
        b._bic_zc_sitel2 site_group_l2,
        b._bic_zc_sitel3 site_group_l3,
        b._bic_zc_subrg1 sub_region_1,
        c.vm_region,
        b._bic_zc_kpists kpi_flag,
        CASE
            WHEN b._bic_zk_saqty < 1000 THEN 'A'
            WHEN b._bic_zk_saqty >= 1000 AND b._bic_zk_saqty < 1500 THEN 'B'
            WHEN b._bic_zk_saqty >= 1500 THEN 'C'
        ELSE NULL
        END size_st,
        CASE
            WHEN b._bic_zk_saqty < 1000 THEN '< 1.000 m2'
            WHEN b._bic_zk_saqty >= 1000 AND b._bic_zk_saqty < 1500 THEN '1.000 m2 - <1.500 m2'
            WHEN b._bic_zk_saqty >= 1500 THEN '>= 1.500 m2'
        ELSE NULL
        END sales_area_text,
        c.ebitda_group,
        c.hub_hang_kho,
        a.status store_status_desc,
        a.comment,
        a.last_updated_date,
        a.store_type
    FROM store_wp_all a
    LEFT JOIN store_sap b ON a.store_id = b.plant
    LEFT JOIN store_operation c ON a.store_id = c.store_id
    LEFT JOIN supply_store_fc b1 ON a.store_id = b1.store_id
    LEFT JOIN dc_fc b2 ON b1.supply_store = b2.store_id
)
,dc AS (
    SELECT
        a.plant store_id,
        a.store_name,
        a.store_name store_description,
        rt_lopdat store_open_date,
        rt_lcldat store_closed_date,
        a.segment as business_unit,
        a.region,
        a.region region_vn,
        a.city city_vn,
        '' district,
        '' ward,
        a.address,
        a.addr_numbr,
        '' sap_pc,
        '' cua_hang_truong,
        '' sdt_cht,
        '' sdt_ch,
        '' email_ch,
        '' qlkv,
        '' sdt_qlkv,
        '' email_qlkv,
        '' gdv,
        '' email_gdv,
        '' gdm,
        '' email_gdm,
        CAST(a.latitude AS STRING) latitude,
        CAST(a.longitude AS STRING) longtitude,
        '' ground,
        '' front_m,
        CAST(_bic_zk_reaqty AS STRING) area_m2,
        CAST(_bic_zk_saqty AS STRING) biz_area_m2,
        CAST(_bic_zc_catsiz AS STRING) size_v,
        '' assortment_v,
        '' concept_id,
        '' concept,
        a.sap_concept,
        '' warehouse_size,
        '' lease_term,
        a.comp_code,
        a.salesorg sales_region,
        a.sales_dist sales_district,
        a.distr_chan distribution_channel,
        a._bic_zc_sup_rg supply_region,
        a._bic_zc_rptsgm reporting_business_unit,
        a._bic_zk_saqty sales_area,
        a._bic_zk_reaqty rent_area,
        a._bic_zk_renamt rent_amount,
        a._bic_zc_region reporting_region,
        a._bic_zc_stoloc store_cat,
        a._bic_zc_catsiz store_cat_size,
        a._bic_zc_sitel2 site_group_l2,
        a._bic_zc_sitel3 site_group_l3,
        a._bic_zc_subrg1 sub_region_1,
        '' vm_region,
        a._bic_zc_kpists kpi_flag,
        CASE
            WHEN a._bic_zk_saqty < 1000 THEN 'A'
            WHEN a._bic_zk_saqty >= 1000 AND a._bic_zk_saqty < 1500 THEN 'B'
            WHEN a._bic_zk_saqty >= 1500 THEN 'C'
        ELSE NULL
        END size_st,
        CASE
            WHEN a._bic_zk_saqty < 1000 THEN '< 1.000 m2'
            WHEN a._bic_zk_saqty >= 1000 AND a._bic_zk_saqty < 1500 THEN '1.000 m2 - <1.500 m2'
            WHEN a._bic_zk_saqty >= 1500 THEN '>= 1.500 m2'
        ELSE NULL
        END sales_area_text,
        '' ebitda_group,
        '' hub_hang_kho,
        IF(store_name LIKE '[Block]%' AND store_name LIKE '%Adayroi%', 'Đóng cửa', 'Đang hoạt động') store_status_desc,
        '' comment,
        a.last_updated_date
    FROM store_sap a
    WHERE a.segment = '1100'
),
store_missing AS (
    SELECT
        b.plant store_id,
        b.store_name,
        b.store_name store_description,
        b.rt_lopdat store_open_date,
        b.rt_lcldat store_closed_date,
        b.segment business_unit,
        b.region,
        '' region_vn,
        b.city city_vn,
        '' district,
        '' ward,
        b.address address,
        b.addr_numbr,
        '' sap_pc,
        '' cua_hang_truong,
        '' sdt_cht,
        '' sdt_ch,
        '' email_ch,
        '' qlkv,
        '' sdt_qlkv,
        '' email_qlkv,
        '' gdv,
        '' email_gdv,
        '' gdm,
        '' email_gdm,
        CAST(b.latitude AS STRING) latitude,
        CAST(b.longitude AS STRING) longtitude,
        '' ground,
        '' front_m,
        CAST(b._bic_zk_reaqty AS STRING) area_m2,
        CAST(b._bic_zk_saqty AS STRING) biz_area_m2,
        b._bic_zc_catsiz size_v,
        '' assortment_v,
        '' concept_id,
        '' concept,
        b.sap_concept,
        '' warehouse_size,
        '' lease_term,
        b.comp_code,
        b.salesorg sales_region,
        b.sales_dist sales_district,
        b.distr_chan distribution_channel,
        b._bic_zc_sup_rg supply_region,
        b._bic_zc_rptsgm reporting_business_unit,
        b._bic_zk_saqty sales_area,
        b._bic_zk_reaqty rent_area,
        b._bic_zk_renamt rent_amount,
        b._bic_zc_region reporting_region,
        b._bic_zc_stoloc store_cat,
        b._bic_zc_catsiz store_cat_size,
        b._bic_zc_sitel2 site_group_l2,
        b._bic_zc_sitel3 site_group_l3,
        b._bic_zc_subrg1 sub_region_1,
        '' vm_region,
        b._bic_zc_kpists kpi_flag,
        CASE
            WHEN b._bic_zk_saqty < 1000 THEN 'A'
            WHEN b._bic_zk_saqty >= 1000 AND b._bic_zk_saqty < 1500 THEN 'B'
            WHEN b._bic_zk_saqty >= 1500 THEN 'C'
        ELSE NULL
        END size_st,
        CASE
            WHEN b._bic_zk_saqty < 1000 THEN '< 1.000 m2'
            WHEN b._bic_zk_saqty >= 1000 AND b._bic_zk_saqty < 1500 THEN '1.000 m2 - <1.500 m2'
            WHEN b._bic_zk_saqty >= 1500 THEN '>= 1.500 m2'
        ELSE NULL
        END sales_area_text,
        '' ebitda_group,
        '' hub_hangkho,
        'Missing' store_status_desc,
        '' comment,
        b.last_updated_date
    FROM store_sap b
        LEFT JOIN store_wp_all a ON a.store_id = b.plant
        LEFT JOIN dc_fc c ON c.store_id = b.plant
    WHERE a.store_id IS NULL AND c.store_id IS NULL AND b.segment IN ('1500', '2000') AND b.store_name NOT LIKE '[Block]%'
),
store_dw AS (
    SELECT
        b.plant store_id,
        b.store_name,
        b.store_name store_description,
        b.rt_lopdat  store_open_date,
        b.rt_lcldat store_closed_date,
        b.segment abusiness_unit,
        b.region,
        '' region_vn,
        b.city city_vn,
        '' district,
        '' ward,
        b.address address,
        b.addr_numbr,
        '' sap_pc,
        '' cua_hang_truong,
        '' sdt_cht,
        '' sdt_ch,
        '' email_ch,
        '' qlkv,
        '' sdt_qlkv,
        '' email_qlkv,
        '' gdv,
        '' email_gdv,
        '' gdm,
        c.email_gdm email_gdm,
        CAST(b.latitude AS STRING) latitude,
        CAST(b.longitude AS STRING) longtitude,
        '' ground,
        '' front_m,
        CAST(b._bic_zk_reaqty AS STRING) area_m2,
        CAST(b._bic_zk_saqty AS STRING) biz_area_m2,
        b._bic_zc_catsiz size_v,
        '' assortment_v,
        '' concept_id,
        '' concept,
        b.sap_concept,
        '' warehouse_size,
        '' lease_term,
        b.comp_code,
        b.salesorg sales_region,
        b.sales_dist sales_district,
        b.distr_chan distribution_channel,
        b._bic_zc_sup_rg supply_region,
        b._bic_zc_rptsgm reporting_business_unit,
        b._bic_zk_saqty sales_area,
        b._bic_zk_reaqty rent_area,
        b._bic_zk_renamt rent_amount,
        b._bic_zc_region reporting_region,
        b._bic_zc_stoloc store_cat,
        b._bic_zc_catsiz store_cat_size,
        b._bic_zc_sitel2 site_group_l2,
        b._bic_zc_sitel3 site_group_l3,
        b._bic_zc_subrg1 sub_region_1,
        '' vm_region,
        b._bic_zc_kpists kpi_flag,
        CASE
            WHEN b._bic_zk_saqty < 1000 THEN 'A'
            WHEN b._bic_zk_saqty >= 1000 AND b._bic_zk_saqty < 1500 THEN 'B'
            WHEN b._bic_zk_saqty >= 1500 THEN 'C'
        ELSE NULL
        END size_st,
        CASE
            WHEN b._bic_zk_saqty < 1000 THEN '< 1.000 m2'
            WHEN b._bic_zk_saqty >= 1000 AND b._bic_zk_saqty < 1500 THEN '1.000 m2 - <1.500 m2'
            WHEN b._bic_zk_saqty >= 1500 THEN '>= 1.500 m2'
        ELSE NULL
        END sales_area_text,
        '' ebitda_group,
        '' hub_hangkho,
        CASE
            WHEN c.store_status_desc = 'Open' THEN 'Đang hoạt động'
            WHEN c.store_status_desc = 'Closed' THEN 'Đóng cửa'
            ELSE NULL
        END store_status_desc,
        '' comment,
        b.last_updated_date,
        CAST(NULL AS STRING) store_type,
        a1.store_id ref_dw_store
    FROM store_sap b
		LEFT JOIN (
			SELECT a.addr_numbr, b.oi_sort2 store_id FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_pcustomer a
				JOIN {catalog_name}.udp_wcm_silver_sap_bw.sap_pzc_add_cl b
				ON a.addr_numbr = b._bic_zc_add_cl
			) a1 ON LTRIM('0', b.addr_numbr) = LTRIM('0', a1.addr_numbr)
		LEFT JOIN store_wp_all a ON a1.store_id = a.store_id
		LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_portal_profile_store_drwin c ON b.plant = c.store_id
    WHERE b.segment = '1009' AND b.store_name NOT LIKE '[Block]%'
)
,STORE_FC AS (
    SELECT
        a.store_id_fc store_id,
        b.store_name,
        b.store_description,
        b.store_open_date,
        b.store_closed_date,
        '6000' AS business_unit,
        b2.region,
        b.region_vn,
        b.city_vn,
        b.district,
        b.ward,
        b.address address,
        b.addr_numbr,
        b.sap_pc,
        b.cua_hang_truong,
        b.sdt_cht,
        b.sdt_ch,
        b.email_ch,
        b.qlkv,
        b.sdt_qlkv,
        b.email_qlkv,
        b.gdv,
        b.email_gdv,
        b.gdm,
        b.email_gdm,
        b.latitude,
        b.longtitude,
        b.ground,
        b.front_m,
        b.area_m2,
        b.biz_area_m2,
        b.size_v,
        b.assortment_v,
        b.concept_id,
        b.concept,
        b.sap_concept,
        b.warehouse_size,
        b.lease_term,
        b.comp_code,
        b.sales_region,
        b.sales_district,
        b.distribution_channel,
        b2.supply_region,
        b.reporting_business_unit,
        b.sales_area,
        b.rent_area,
        b.rent_amount,
        b.reporting_region,
        b.store_cat,
        b.store_cat_size,
        b.site_group_l2,
        b.site_group_l3,
        b.sub_region_1,
        b.vm_region,
        b.kpi_flag,
        b.size_st,
        b.sales_area_text,
        b.ebitda_group,
        '' hub_hangkho,
        IF(a.store_id_fc IN ('F203','F202'),'Đang hoạt động', 'Sắp khai trương') store_status_desc,
        b.comment,
        b.last_updated_date,
        b.store_id ref_fc_store
    FROM store b
    JOIN (
            SELECT a.plant store_id, b.oi_sort1 store_id_fc FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_pcustomer a JOIN {catalog_name}.udp_wcm_silver_sap_bw.sap_pzc_add_cl b
            ON a.addr_numbr = b._bic_zc_add_cl WHERE a.addr_numbr IS NOT NULL AND REGEXP_LIKE(b.oi_sort1,r'F[0-9]')
            AND a.accnt_grp = 'Z110'
        ) a USING (store_id)
    LEFT JOIN supply_store_fc b1 ON a.store_id_fc = b1.store_id
    LEFT JOIN dc_fc b2 ON b1.supply_store = b2.store_id
)
,STORE_FULL AS (
    SELECT *, CAST(NULL AS STRING) store_type, CAST(NULL AS STRING) ref_dw_store, CAST(NULL AS STRING) ref_fc_store, CAST(NULL AS STRING) supply_store, CAST(NULL AS STRING) ref_fc_code
        FROM dc
    UNION ALL
    SELECT a.*, CAST(NULL AS STRING) ref_dw_store, CAST(NULL AS STRING) ref_fc_store, b.supply_store, b.ref_fc_code
        FROM store a LEFT JOIN supply_store_fc b USING (store_id)
    UNION ALL
    SELECT a.*, CAST(NULL AS STRING) store_type, CAST(NULL AS STRING) ref_dw_store, CAST(NULL AS STRING) ref_fc_store, b.supply_store, b.ref_fc_code
        FROM store_missing a LEFT JOIN supply_store_fc b USING (store_id)
    UNION ALL
    SELECT *, CAST(NULL AS STRING) store_type, CAST(NULL AS STRING) ref_dw_store, CAST(NULL AS STRING) ref_fc_store, CAST(NULL AS STRING) supply_store, CAST(NULL AS STRING) ref_fc_code
        FROM dc_fc
    UNION ALL
	SELECT *, CAST(NULL AS STRING) ref_fc_store, CAST(NULL AS STRING) supply_store, CAST(NULL AS STRING) ref_fc_code
        FROM store_dw
	UNION ALL
    SELECT a.* EXCEPT(ref_fc_store), CAST(NULL AS STRING) store_type, CAST(NULL AS STRING) ref_dw_store, a.ref_fc_store, b.supply_store, b.ref_fc_code
        FROM store_fc a LEFT JOIN supply_store_fc b USING (store_id)
),
MAIN_1 AS (
    SELECT
        a.*,
        c.reporting_region_desc mien,
        b.region_code_desc region_description,
        c.reporting_region_name region_domain,
        c.reporting_region_desc region_domain_vn,
        CASE
            WHEN store_status_desc = 'Đang hoạt động' THEN 'OPEN'
            WHEN store_status_desc = 'Đóng cửa' THEN 'CLOSED'
            WHEN store_status_desc = 'Sắp đóng cửa' OR store_status_desc = 'Sắp khai trương' THEN 'OPEN'
            WHEN store_status_desc = 'Tạm dừng reopening' THEN 'DELAYED_REOPENING'
            ELSE 'OTHER'
        END store_status_id
    FROM (
        SELECT *,
            CASE
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'HCM' THEN 'HỒ CHÍ MINH'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'HỒ CHÍ MINH' THEN 'HỒ CHÍ MINH'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'THỪA THIÊN - HUẾ' THEN 'THỪA THIÊN HUẾ'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'HUẾ' THEN 'THỪA THIÊN HUẾ'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'ĐẮK LẮK' THEN 'DAKLAK'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'THANH HOÁ' THEN 'THANH HÓA'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'NHA TRANG' THEN 'KHÁNH HÒA'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'BÀ RỊA – VŨNG TÀU' THEN 'BÀ RỊA - VŨNG TÀU'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'BÀ RỊA VŨNG TÀU' THEN 'BÀ RỊA - VŨNG TÀU'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'BÀ RỊA' THEN 'BÀ RỊA - VŨNG TÀU'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'BÀ RỊA-VŨNG TÀU' THEN 'BÀ RỊA - VŨNG TÀU'
                WHEN TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', '')) = 'THỦ DẦU MỘT BÌNH DƯƠNG' THEN 'BÌNH DƯƠNG'
            ELSE TRIM(REGEXP_REPLACE(UPPER(REPLACE(CITY_VN,',','')), 'TỈNH |T. |TP. |TP.|THÀNH PHỐ ', ''))
            END CITY
        FROM store_full
    ) a
    LEFT JOIN (SELECT DISTINCT region_code_desc, region region_code FROM {catalog_name}.udp_wcm_silver_dim.region) b ON LOWER(a.city) = LOWER(b.region_code_desc)
    LEFT JOIN {catalog_name}.udp_wcm_silver_dim.report_region c ON b.region_code = c.reporting_region_desc
),
MAIN_2 AS (
    SELECT
        *,
        IF(store_id BETWEEN '6A00' AND '6Z99', 'YES', 'NO') is_win_plus
    FROM main_1 a
    WHERE a.store_name IS NOT NULL
    QUALIFY ROW_NUMBER() OVER (PARTITION BY store_id ORDER BY last_updated_date DESC) = 1
),
store_open_date_earliest as (
   select store_id, min(store_open_date) store_open_date_earliest from {catalog_name}.udp_wcm_gold_vcm_dwh.d_store_hist group by store_id 
)
SELECT
  a.*,
  b._bic_zc_region sap_region,
  CASE b._bic_zc_region
  WHEN '01' THEN 'Miền Bắc'
  WHEN '02' THEN 'Miền Nam'
  WHEN '03' THEN 'Miền Trung'
  WHEN '04' THEN 'Miền Tây'
  ELSE NULL END sap_region_desc,
  CASE
  WHEN store_name LIKE '[Block]%' THEN 'YES'
  WHEN UPPER(store_name) like '%DO NOT USE%' THEN 'YES'
  ELSE 'NO' END block_status,
c.store_open_date_earliest,
CASE
  WHEN c.store_open_date_earliest < ADD_MONTHS(DATE_TRUNC('YEAR', DATE(CURRENT_TIMESTAMP + INTERVAL 7 HOURS)), -12) THEN 'LFL'
  WHEN c.store_open_date_earliest >=  ADD_MONTHS(DATE_TRUNC('YEAR', DATE(CURRENT_TIMESTAMP + INTERVAL 7 HOURS)), -12) 
       AND c.store_open_date_earliest < DATE_TRUNC('YEAR', DATE(CURRENT_TIMESTAMP + INTERVAL 7 HOURS)) THEN 'STU'
  WHEN c.store_open_date_earliest >= DATE_TRUNC('YEAR', DATE(CURRENT_TIMESTAMP + INTERVAL 7 HOURS)) THEN 'NSO'
END AS panel
FROM main_2 a
LEFT JOIN (
    SELECT * EXCEPT (rn) FROM (
      SELECT plant, _bic_zc_region, ROW_NUMBER() OVER (PARTITION BY plant ORDER BY proc_date DESC) rn FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_0plant
    ) WHERE rn = 1
) b
ON a.store_id = b.plant
LEFT JOIN store_open_date_earliest c using (store_id)

""")